$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
$$

# Part 3: Mini-Project
<a id=part3></a>

In this part you'll implement a small comparative-analysis project, heavily based on the materials from the tutorials and homework.

You must **choose one** of the project options specified below.

### Guidelines

- You should implement the code which displays your results in this notebook, and add any additional code files for your implementation in the `project/` directory. You can import these files here, as we do for the homeworks.
- Running this notebook should not perform any training - load your results from some output files and display them here. The notebook must be runnable from start to end without errors.
- You must include a detailed write-up (in the notebook) of what you implemented and how. 
- Explain the structure of your code and how to run it to reproduce your results.
- Explicitly state any external code you used, including built-in pytorch models and code from the course tutorials/homework.
- Analyze your numerical results, explaining **why** you got these results (not just specifying the results).
- Where relevant, place all results in a table or display them using a graph.
- Before submitting, make sure all files which are required to run this notebook are included in the generated submission zip.
- Try to keep the submission file size under 10MB. Do not include model checkpoint files, dataset files, or any other non-essentials files. Instead include your results as images/text files/pickles/etc, and load them for display in this notebook. 

## Sentiment Analysis with Self-Attention and Word Embeddings

Based on Tutorials 6 and 7, we'll implement and train an improved sentiment analysis model.
We'll use self-attention instead of RNNs and incorporate pre-trained word embeddings.

In tutorial 6 we saw that we can train word embeddings together with the model.
Although this produces embeddings which are customized to the specific task at hand,
it also greatly increases training time.
A common technique is to use pre-trained word embeddings.
This is essentially a large mapping from words (e.g. in english) to some
high-dimensional vector, such that semantically similar words have an embedding that is
"close" by some metric (e.g. cosine distance).
Use the [GloVe](https://nlp.stanford.edu/projects/glove/) 6B embeddings for this purpose.
You can load these vectors into the weights of an `nn.Embedding` layer.

In tutorial 7 we learned how attention can be used to learn to predict a relative importance
for each element in a sequence, compared to the other elements.
Here, we'll replace the RNN with self-attention only approach similar to Transformer models, roughly based on [this paper](https://www.aclweb.org/anthology/W18-6219.pdf).
After embedding each word in the sentence using the pre-trained word-embedding a positional-encoding vector is added to provide each word in the sentence a unique value based on it's location.
One or more self-attention layers are then applied to the results, to obtain an importance weighting for each word.
Then we classify the sentence based on the average these weighted encodings.


Now, using these approaches, you need to:

- Implement a **baseline** model: Use pre-trained embeddings with an RNN-based model.
You can use LSTM/GRU or bi-directional versions of these, in a way very similar to what we implemented in the tutorial.
-  Implement an **improved** model: Based on the self-attention approach, implement an attention-based sentiment analysis model that has 1-2 self-attention layers instead of an RNN. You should use the same pre-trained word embeddings for this model.
- You can use pytorch's built-in RNNs, attention layers, etc.
- For positional encoding you can use the sinosoidal approach described in the paper (first proposed [here](https://arxiv.org/pdf/1706.03762.pdf)). You can use existing online implementations (even though it's straightforward to implement). 
- You can use the SST database as shown in the tutorial.

Your results should include:
- Everything written in the **Guidelines** above.
- A comparative analysis: compare the baseline to the improved model. Compare in terms of overall classification accuracy and show a multiclass confusion matrix.
- Visualize of the attention maps for a few movie reviews from each class, and explain the results.

In [1]:
# Setup
%matplotlib inline
!pip install torchsummary
from torchsummary import summary

import os
import sys
import time
import torch
import matplotlib.pyplot as plt
import warnings
import torch.nn as nn
import numpy as np
warnings.simplefilter("ignore")
plt.rcParams['font.size'] = 20
data_dir = os.path.expanduser('~/.pytorch-datasets')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

**Dataset**<br/>
We would like to test a recurrent model over varios datasets.<br/>
We use torchtext package to load SST dataset (3 classes, like recitation and 5 classes, like the paper linked above).<br/>
We decided to use bucket iterator, as presented in the recitation, since it minimize pad tokens and popular in NLP projects.

In [2]:
import torchtext.data
from torchtext.vocab import Vectors, GloVe
import torchtext.datasets
BATCH_SIZE = 32

def load_dataset(name='SST', classes=3):
    review_parser = torchtext.data.Field(
        sequential=True, use_vocab=True, lower=True,
        init_token='<sos>', eos_token='<eos>', dtype=torch.long,
        tokenize='spacy', tokenizer_language='en_core_web_sm')
    # This Field object converts the text labels into numeric values (0,1,2)
    label_parser = torchtext.data.Field(
        is_target=True, sequential=False, unk_token=None, use_vocab=True
    )

    if name=='SST':
        fine_grained = False
        if classes==5:
            fine_grained = True
            
        #3 classes SST
        ds_train, ds_valid, ds_test = torchtext.datasets.SST.splits(
            review_parser, label_parser, root=data_dir,fine_grained=fine_grained)

        print(f'Number of training samples: {len(ds_train)}')
        print(f'Number of test     samples: {len(ds_test)}')
    #build the vocabulary:
    review_parser.build_vocab(ds_train,vectors=GloVe(name='6B', dim=300))
    label_parser.build_vocab(ds_train)
    word_embeddings = review_parser.vocab.vectors
    word_embeddings = word_embeddings.to(device=device)
        
        
    # BucketIterator creates batches with samples of similar length
    # to minimize the number of <pad> tokens in the batch.
    dl_train, dl_valid, dl_test = torchtext.data.BucketIterator.splits((ds_train, ds_valid, ds_test), batch_size=BATCH_SIZE,shuffle=True, device=device)
    train_iter, valid_iter, test_iter = torchtext.data.BucketIterator.splits((ds_train, ds_valid, ds_test), batch_size=BATCH_SIZE, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)
    return word_embeddings,train_iter, valid_iter, test_iter

In [3]:
word_embeddings,train_iter, valid_iter, test_iter = load_dataset(name='SST', classes=3)
word_embeddings_5,train_iter_5, valid_iter_5, test_iter_5 = load_dataset(name='SST', classes=5)

Number of training samples: 8544
Number of test     samples: 2210
Number of training samples: 8544
Number of test     samples: 2210


## Baseline Model:
For the implementation of the sentiment analysis basic model, we used bidirectional GRU,<br/>
the reason we chose it is to avoid the vanishing gradients problem that accure when one might use a simple RNN.<br/>

Lets load the model and show the architecture summary. note that all architectures of network are implemented in the project directory.<br/>
The hyper-parameters were tuned using a grid search in the notebook Hype_Parameters.ipynb (batch size, hidden dim, num of layers and lr).<br/>
Embedding dimension is predefined by GloVe model, the input dimension is dependant of the dataset as well as the output dimension.

In [4]:
batch = next(iter(train_iter))
X, y = batch.text, batch.label
X.shape

torch.Size([41, 32])

In [5]:
from project.RNN import SentimentAnalyzer
from project.Attention import AttentionAnalyzer,MHA

VOC_DIM = word_embeddings.shape[0]
EMBEDDING_DIM = 300
HIDDEN_DIM = 16 #
OUTPUT_DIM = 3,5 #5
NUM_LAYERS = 1
#We WILL instantiate two models, one for the fine grained dataset and one for the 3-classes dataset
model3 = SentimentAnalyzer(VOC_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM[0], word_embeddings, layers=NUM_LAYERS,dropout=0.3).to(device)
model5 = SentimentAnalyzer(VOC_DIM, EMBEDDING_DIM, HIDDEN_DIM*4, OUTPUT_DIM[1], word_embeddings_5, layers=NUM_LAYERS*2,dropout=0.3).to(device)

print(f"Baseline model (3-classes):{model3}")

Baseline model (3-classes):SentimentAnalyzer(
  (embd): Embedding(15482, 300)
  (rnn): GRU(300, 16, dropout=0.3, bidirectional=True)
  (sentiment): Linear(in_features=32, out_features=3, bias=True)
  (DO): Dropout(p=0.3, inplace=False)
  (log_softmax): LogSoftmax(dim=1)
)


The model contain pre-trained embedding, bidirectional GRU as RNN, then a FC layer from the hidden dimantion to the output, and a  log softmax layer for logits (so we can use NLLL loss later on).<br/>
We also used Dropout in the GRE and in the FC to avoid some of the overfitting.<br/>

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The Baseline model for SST-3 has {count_parameters(model3):,} trainable weights, and the Baseline model for SST-5 has {count_parameters(model5):,} trainable weights.')

The Baseline model for SST-3 has 30,627 trainable weights, and the Baseline model for SST-5 has 215,685 trainable weights.


Defining the hyperparameters and checkpoints file for the training of the model:

In [7]:

checkpoint_attn =''# 'project/Models/ATTN'
learning_rate = 1e-3
batch_size = BATCH_SIZE
hidden_size = HIDDEN_DIM
embedding_length = EMBEDDING_DIM
EPOCHS = 20

In [8]:
from project.Trainer import train_and_eval, present_accuracy,print_stats

***Training the baseline model over the SST-3 dataset***

In [9]:
optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
# Train, unless final checkpoint is found
checkpoint = 'project/Models/SST3_RNN_no'
checkpoint_file_final = f'{checkpoint}.pt'
if os.path.isfile(checkpoint_file_final):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} (not training)')
    saved_state = torch.load(checkpoint_file_final, map_location=device)
    model3.load_state_dict(saved_state['model_state'])
else:
    try: 
        train_accur, test_accur = train_and_eval(model3,train_iter, test_iter, optimizer, loss_fn=loss_fn, epochs=EPOCHS,name='RNN_',data_name = 'SST3')
    except KeyboardInterrupt as e:
        print('\n *** Training interrupted by user')


Epoch #0,train loss=1.060,train accuracy=0.414, validation accuracy=0.405, elapsed=2.0 sec
*** Saved checkpoint project/Models/SST3_RNN_.pt at epoch 1
Epoch #1,train loss=1.059,train accuracy=0.414, validation accuracy=0.416, elapsed=1.6 sec
*** Saved checkpoint project/Models/SST3_RNN_.pt at epoch 2
Epoch #2,train loss=1.057,train accuracy=0.417, validation accuracy=0.518, elapsed=1.5 sec
*** Saved checkpoint project/Models/SST3_RNN_.pt at epoch 3
Epoch #3,train loss=0.988,train accuracy=0.524, validation accuracy=0.622, elapsed=1.6 sec
*** Saved checkpoint project/Models/SST3_RNN_.pt at epoch 4
Epoch #4,train loss=0.874,train accuracy=0.612, validation accuracy=0.646, elapsed=1.6 sec
*** Saved checkpoint project/Models/SST3_RNN_.pt at epoch 5
Epoch #5,train loss=0.816,train accuracy=0.640, validation accuracy=0.669, elapsed=1.6 sec
*** Saved checkpoint project/Models/SST3_RNN_.pt at epoch 6
Epoch #6,train loss=0.784,train accuracy=0.656, validation accuracy=0.664, elapsed=1.6 sec
Epo

ValueError: too many values to unpack (expected 2)

In [ ]:
present_accuracy(model3, test_iter,classes=3);

Here we can see the final accuracy, alongside with the confusion matrix <br/><br/>

The model is slightly overfitting, as the trainin accuracy is higher then the test accuracy.<br/>
Alongside with that, the confusion matrix show us that the neutral class have the most mistakes, while for positive and negative the model is pretty accurate, it does not know how to classify the neutral examples.<br/>
This phenomenan is familiar and make sence for two reasons.
First, even for human being it is hard to say when something is nither good of bad.<br/>
Second, the number of the neutral samples is about half as the number of positive and negative, and it's usually happen with imbalanced data.<br/>

to support this claim, we provide some more statistics below, alongside with Precision, Recall and F1 score
and show that the F1 for neutral is extreamly low compare to the rest of the model accuracy.

In [ ]:
print_stats(model3, test_iter,classes=3)

One might change the loss function to optimize over F1 or over the neutral class if it's important.
Since we train a model with no 'real-life' perpuse, we optimize only to achive better accuracy and leave the other options open for other work.

## Training on SST-5:
We will now train the same model on the fine grained SST to see how it performs on a more difficult task

In [ ]:
checkpoint = 'project/Models/SST5_RNN_no'
optimizer = torch.optim.Adam(model5.parameters(), lr=learning_rate)
checkpoint_file_final = f'{checkpoint}.pt'
if os.path.isfile(checkpoint_file_final):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} (not training)')
    saved_state = torch.load(checkpoint_file_final, map_location=device)
    model5.load_state_dict(saved_state['model_state'])
else:
    try: 
        train_accur, test_accur = train_and_eval(model5,train_iter_5, test_iter_5, optimizer, loss_fn=loss_fn, epochs=EPOCHS,name='RNN_',data_name = 'SST5')
    except KeyboardInterrupt as e:
        print('\n *** Training interrupted by user')

In [ ]:
present_accuracy(model5, test_iter_5,classes=5)

Here we show the accuracy and confusion matrix as before.<br/><br/>

This is a much harder problem to solve, as we can see from the confusion matrix, the model is the most certain about the positive and negative classes.

We would also show the same statistical attributes as before

In [ ]:
print_stats(model5, test_iter_5,classes=5)

## Self Attention Model
We've tried to overcome this results by using a self attention mechanisem.<br/>

In this work we've implemented both the given paper self attention and a sligthly newer version that we've found that work better for the given problem.<br/>

This model has an attention layer, ontop of the GRU baed rnn, so the representation of each token hidden state is multiply with the Attention head (that in turn, get the rnn output as an input for the Query, Key and Value, thus the name self attention).<br/>

The first implementation based on "Self-Attention-Based BiLSTM Model for Short Text Fine-Grained Sentiment Classification" of xie et el since it achive better results<br/>.

ilustration:
<img src ="Attention"/>

In [ ]:
attnModel = AttentionAnalyzer(VOC_DIM, EMBEDDING_DIM, HIDDEN_DIM*2, OUTPUT_DIM[0], word_embeddings, layers=NUM_LAYERS,dropout=0.4).to(device) #attention model for SST3
print(f'The Attention model for 3-classes has {count_parameters(attnModel):,} trainable weights')

***Training the model with attention for SST-3***

In [ ]:
optimizer = torch.optim.Adam(attnModel.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
# Train, unless final checkpoint is found
checkpoint_attn = 'project/Models/SST3_Attention_'
checkpoint_file_final = f'{checkpoint_attn}.pt'
if os.path.isfile(checkpoint_file_final):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} (not training)')
    saved_state = torch.load(checkpoint_file_final, map_location=device)
    attnModel.load_state_dict(saved_state['model_state'])
else:
    try:
        train_and_eval(attnModel,train_iter, valid_iter, optimizer, loss_fn=loss_fn, epochs=EPOCHS,name='Attention_',data_name = 'SST3')

    except KeyboardInterrupt as e:
        print('\n *** Training interrupted by user')

In [ ]:
present_accuracy(attnModel, test_iter,classes=3)
print_stats(attnModel, test_iter,classes=3)

As we can see, this model still strugle with the neutral class, but achive better results in about 2-3 %

In [ ]:
attnModel5 = AttentionAnalyzer(VOC_DIM,EMBEDDING_DIM,HIDDEN_DIM*8,OUTPUT_DIM[1],word_embeddings_5,layers=NUM_LAYERS*2,dropout=0.4).to(device)#attention model for SST5
print(f"Attention model:{attnModel}")
print(f'The Attention model for 5-classes has {count_parameters(attnModel5):,} trainable weights')

***Training the attention model for STT-5***

In [ ]:
checkpoint_attn ='project/Models/Attention_'
optimizer = torch.optim.Adam(attnModel5.parameters(), lr=learning_rate)
# Train, unless final checkpoint is found
checkpoint_file_final = f'{checkpoint_attn}.pt'
if os.path.isfile(checkpoint_file_final):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} (not training)')
    saved_state = torch.load(checkpoint_file_final, map_location=device)
    attnModel5.load_state_dict(saved_state['model_state'])
else:
    try:
        train_and_eval(attnModel5,train_iter_5, test_iter_5, optimizer, loss_fn=loss_fn, epochs=EPOCHS,name='Attention_',data_name = 'SST5')

    except KeyboardInterrupt as e:
        print('\n *** Training interrupted by user')

In [ ]:
present_accuracy(attnModel5, test_iter_5,classes=5)
print_stats(attnModel5, test_iter_5,classes=5)

## Multi Head Attention
<br/>

Now we would provide a heavier model, that uses N attention heads (we chose 8) that work on the word embeding and Avg the score over them.<br/>
This is the score of all the heads of the sentence, and then we use 2 layers of FC to reduce and make the prediction.<br/>

<img src='Multihead'/>

In [ ]:
MHA = MHAA(VOC_DIM, EMBEDDING_DIM, HIDDEN_DIM*2, OUTPUT_DIM[0], word_embeddings, layers=NUM_LAYERS,dropout=0.4).to(device) #attention model for SST3
print(f'The Multy Head Attention model for 3-classes has {count_parameters(MHA):,} trainable weights')
optimizer = torch.optim.Adam(MHA.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
train_and_eval(MHA,train_iter, test_iter, optimizer, loss_fn=loss_fn, epochs=EPOCHS,name='MHA',data_name = 'SST3');
present_accuracy(MHA, test_iter,classes=3)
print_stats(MHA, test_iter,classes=3)

In [ ]:
MHA5 = MHAA(VOC_DIM, EMBEDDING_DIM, HIDDEN_DIM*2, OUTPUT_DIM[1], word_embeddings, layers=NUM_LAYERS,dropout=0.4).to(device) #attention model for SST3
print(f'The Multy Head Attention model for 5-classes has {count_parameters(MHA):,} trainable weights')
optimizer = torch.optim.Adam(MHA5.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
train_accur, test_accur,train_losses,test_losses = train_and_eval(MHA5,train_iter, test_iter, optimizer, loss_fn=loss_fn, epochs=EPOCHS,name='MHA',data_name = 'SST3');
present_accuracy(MHA5, test_iter,classes=5)
print_stats(MHA5, test_iter,classes=5)

## Spectrally-Normalized Wasserstein GANs

In HW3 we implemented a simple GANs from scratch, using an approach very similar to the original GAN paper. However, the results left much to be desired and we discovered first-hand how hard it is to train GANs due to their inherent instability.

One of the prevailing approaches for improving training stability for GANs is to use a technique called [Spectral Normalization](https://arxiv.org/pdf/1802.05957.pdf) to normalize the largest singular value of a weight matrix so that it equals 1.
This approach is generally applied to the discriminator's weights in order to stabilize training. The resulting model is sometimes referred to as a SN-GAN.
See Appendix A in the linked paper for the exact algorithm. You can also use pytorch's `spectral_norm`.

Another very common improvement to the vanilla GAN is known a [Wasserstein GAN](https://arxiv.org/pdf/1701.07875.pdf) (WGAN). It uses a simple modification to the loss function, with strong theoretical justifications based on the Wasserstein (earth-mover's) distance.
See also [here](https://developers.google.com/machine-learning/gan/loss) for a brief explanation of this loss function.

One problem with generative models for images is that it's difficult to objectively assess the quality of the resulting images.
To also obtain a quantitative score for the images generated by each model,
we'll use the [Inception Score](https://arxiv.org/pdf/1606.03498.pdf).
This uses a pre-trained Inception CNN model on the generated images and computes a score based on the predicted probability for each class.
Although not a perfect proxy for subjective quality, it's commonly used a way to compare generative models.
You can use an implementation of this score that you find online, e.g. [this one](https://github.com/sbarratt/inception-score-pytorch) or implement it yourself.

Based on the linked papers, add Spectral Normalization and the Wassertein loss to your GAN from HW3.
Compare between:
- The baseline model (vanilla GAN)
- SN-GAN (vanilla + Spectral Normalization)
- WGAN (using Wasserstein Loss)
- Optional: SN+WGAN, i.e. a combined model using both modifications.

As a dataset, you can use [LFW](http://vis-www.cs.umass.edu/lfw/) as in HW3 or [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html), or even choose a custom dataset (note that there's a dataloder for CelebA in `torchvision`). 

Your results should include:
- Everything written in the **Guidelines** above.
- A comparative analysis between the baseline and the other models. Compare:
  - Subjective quality (show multiple generated images from each model)
  - Inception score (can use a subset of the data).
- You should show substantially improved subjective visual results with these techniques.

## Implementation

**TODO**: This is where you should write your explanations and implement the code to display the results.
See guidelines about what to include in this section.